In [ ]:
!pip install 'crewai[tools]'

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of mem0ai to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other

In [ ]:
! pip install ollama

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13320    0 13320    0     0  14617      0 --:--:-- --:--:-- --:--:-- 14605>>> Installing ollama to /usr/local
100 13320    0 13320    0     0  13819      0 --:--:-- --:--:-- --:--:-- 13817
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama serve > rocama.log 2>&1 &
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 9.1 MB/4

In [ ]:
from crewai import Task, Crew, Process, Agent, LLM
import ollama

# Initialize the local LLaMA 3 model via Ollama
#local_llama3 = ollama.chat(model="llama3")
local_llama3 =LLM(model="ollama/llama3", base_url="http://localhost:11434")

# Define the generator agent with a backstory and detailed parameters
generator = Agent(
    role="Travel Content Creator",
    goal="Generate engaging and relevant travel-related questions for a first-time visitor to Amsterdam",
    backstory="""You are an experienced travel content creator who knows how to generate engaging and insightful questions for first-time travelers.
    You focus on understanding travelers’ needs and are able to create questions that address practical aspects of travel, like transportation, safety, and cultural norms.""",
    verbose=True,
    allow_delegation=False,  # In this case, the agent should not delegate the task
    llm=local_llama3  # Use LLaMA 3 via Ollama as the LLM for this agent
)

# Define the summarizer agent for answering the travel questions
summarizer = Agent(
    role="Travel Expert",
    goal="Provide concise and helpful answers to common questions about visiting Amsterdam",
    backstory="""You are an expert in Amsterdam tourism with deep knowledge of the city’s transport, attractions, and cultural norms.
    You can provide concise, practical answers to travelers' questions that are easy to understand and apply.""",
    verbose=True,
    allow_delegation=False,  # This agent should handle the task independently
    llm=local_llama3  # Use LLaMA 3 for summarizing answers
)


# Define the reviewer agent for reviewing the final output
reviewer = Agent(
    role="Travel Content Reviewer",
    goal="Review the final Markdown document for accuracy, readability, and relevance",
    backstory="""You are a reviewer with extensive experience in travel content. You ensure that the questions and answers are accurate, useful,
    and the formatting is clean. Your goal is to make sure that the content meets the needs of first-time travelers to Amsterdam.""",
    verbose=True,
    allow_delegation=False  # This agent will handle the review independently
)

# Task to generate top 10 questions for a first-time traveler to Amsterdam
task_generate_questions = Task(
    description="Generate a list of the top 10 questions that a first-time traveler might ask about visiting Amsterdam.",
    agent=generator,  # The generator agent is responsible for this task
    expected_output="A list of 10 questions"  # Define what output is expected
)

# Task to summarize answers for each question
task_summarize_answers = Task(
    description="Provide a short and concise answer for each of the top 10 questions about visiting Amsterdam.",
    agent=summarizer,  # The summarizer agent handles this task
    expected_output="A list of short, concise answers"  # Define the expected output for the summary
)

# Task to review the final Markdown output
task_review_output = Task(
    description="Review the final Markdown document for accuracy and readability.",
    agent=reviewer,  # The reviewer agent handles this task
    expected_output="Reviewed and accurate Markdown document"  # Define the expected reviewed document
)

# Instantiate the Crew of agents with their respective tasks
crew = Crew(
    agents=[generator, summarizer, reviewer],  # List of all the agents
    tasks=[task_generate_questions, task_summarize_answers, task_review_output],  # The tasks they will perform sequentially
    verbose=True,  # Set verbosity level for detailed output logs
    process=Process.sequential,  # Process the tasks sequentially
)

# Start the process
result = crew.kickoff()

# Print the final result (the Markdown document)
print(result)


# Agent: Travel Content Creator
## Task: Generate a list of the top 10 questions that a first-time traveler might ask about visiting Amsterdam.


# Agent: Travel Content Creator
## Final Answer: 
1. What are the best ways to get around Amsterdam without a car?
2. Are there any areas in Amsterdam that I should avoid or be cautious of during my trip?
3. How do I navigate the city's bike lanes and traffic rules as a pedestrian?
4. Is it true that most museums in Amsterdam have free admission on certain days of the week?
5. What are some popular neighborhood bars and restaurants that locals frequent?
6. Are there any language barriers I should be aware of, or can I get by with just English?
7. How do I stay safe and secure my belongings while exploring the city's attractions?
8. Can you recommend some budget-friendly accommodations in Amsterdam for a first-time traveler?
9. What are some must-try local foods and drinks that I shouldn't miss during my trip?
10. Are there any special events 

ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable




# Agent: Travel Expert
## Final Answer: 
1. What are the best ways to get around Amsterdam without a car?
The best way to get around Amsterdam without a car is by using public transportation, such as trams, buses, and metro lines. You can also use taxis or ride-hailing services like Uber. Additionally, you can walk or bike through the city's scenic canals and streets.

2. Are there any areas in Amsterdam that I should avoid or be cautious of during my trip?
While Amsterdam is generally a safe city, some areas to be aware of include the Red Light District, which can be seedy at night, and certain neighborhoods with high crime rates, such as Oud-Zuid and Bijlmerburen. Additionally, be cautious when walking alone at night or carrying valuables.

3. How do I navigate the city's bike lanes and traffic rules as a pedestrian?
As a pedestrian in Amsterdam, follow the rules of the road, use crosswalks, and look out for bikes. Be aware that bikes often have priority over pedestrians on sidewal

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable